# Methane - in UKESM, it's a beast


From Fiona's JAMES paper https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2022MS002991, the total 1850-2014 ERF from methane is 0.97 W m-2. This is comprised:

- +0.54 W m-2 direct forcing
- +0.13 W m-2 ozone
- +0.07 W m-2 stWV
- +0.28 W m-2 ACI
- -0.05 W m-2 ARI

In [1]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
from scipy.optimize import root

/Users/c.l.b.alexander/Documents/FAIR/fair_virtual_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [3]:
f = FAIR(temperature_prescribed=True)

In [4]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = [
    'CH4',
    'N2O',
    'CO2',
    'Equivalent effective stratospheric chlorine',
    'VOC',
    'NOx',
    'BC', 
    'OC', 
    'Sulfur', 
    'Ozone',
    'Stratospheric water vapour',
    'Aerosol-radiation interactions', 
    'Aerosol-cloud interactions'
]
species, properties = read_properties(species=species)
# properties = {
#     'CH4': {
#         'type': 'ch4',
#         'input_mode': 'concentration',
#         'greenhouse_gas': True,
#         'aerosol_chemistry_from_emissions': False,
#         'aerosol_chemistry_from_concentration': True,
#     }
# }
properties['CH4']['input_mode'] = 'concentration'
properties['N2O']['input_mode'] = 'concentration'
properties['CO2']['input_mode'] = 'concentration'
properties['Equivalent effective stratospheric chlorine']['input_mode'] = 'concentration'
properties

{'CH4': {'type': 'ch4',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'N2O': {'type': 'n2o',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'CO2': {'type': 'co2',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': False},
 'Equivalent effective stratospheric chlorine': {'type': 'eesc',
  'input_mode': 'concentration',
  'greenhouse_gas': False,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'VOC': {'type': 'other slcf',
  'input_mode': 'emissions',
  'greenhouse_gas': False,
  'aerosol_chemistry_from_emissions': True,
  'aerosol_chemistry_from_concentration': False},
 'NOx': {'type': 'other slcf',
  'input_mode': 'emissions',
  'green

In [5]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(species, properties)

In [6]:
f.allocate()

In [7]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [8]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [9]:
#f.fill_species_configs()

In [10]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [11]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [12]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850'].values[0]
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850'].values[0]
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850'].values[0]
voc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|VOC')&(emis_df['Region']=='World'),'1850'].values[0]
nox = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|NOx')&(emis_df['Region']=='World'),'1850'].values[0]

co2 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CO2')&(conc_df['Region']=='World'),'1850'].values[0]
n2o = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|N2O')&(conc_df['Region']=='World'),'1850'].values[0]
ch4 = conc_df.loc[(conc_df['Scenario']=='ssp245')&(conc_df['Variable']=='Atmospheric Concentrations|CH4')&(conc_df['Region']=='World'),'1850':'2015'].values.squeeze()
# make 2015 equal to 2014
ch4[-1] = ch4[-2]

In [13]:
bc, oc, so2, voc, nox

(np.float64(2.5711244787427185),
 np.float64(18.22682478664064),
 np.float64(4.544463775546904),
 np.float64(66.78304875030415),
 np.float64(13.456576847229837))

In [14]:
# 1850 baselines
f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = so2
f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = bc
f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = oc
f.species_configs['baseline_emissions'].loc[dict(specie='VOC')] = voc
f.species_configs['baseline_emissions'].loc[dict(specie='NOx')] = nox

f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4[0]
f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o
f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2
f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

In [15]:
# fill emissions and concentrations
f.emissions.loc[dict(specie='Sulfur')] = so2
f.emissions.loc[dict(specie='BC')] = bc
f.emissions.loc[dict(specie='OC')] = oc
f.emissions.loc[dict(specie='VOC')] = voc
f.emissions.loc[dict(specie='NOx')] = nox

f.concentration.loc[dict(specie='CH4')] = ch4[:, None, None]
f.concentration.loc[dict(specie='N2O')] = n2o
f.concentration.loc[dict(specie='CO2')] = co2
f.concentration.loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

f.temperature[:] = 0

# pre-calibrated for aerosol-only runs from notebook 05: DON'T ADJUST
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00283793
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.01757433
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.0028512

f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -8.22336281e-01
f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 3.01888722e-02
f.species_configs['aci_shape'].loc[dict(specie='BC')] = 5.07111186e-30
f.species_configs['aci_shape'].loc[dict(specie='OC')] = 6.60381587e-16

In [16]:
# iterative tuning
target_ch4_ch4 = +0.54
target_ch4_o3 = +0.13
target_ch4_stwv = +0.07
target_ch4_ari = -0.05
target_ch4_aci = +0.28

In [17]:
def erf_rootfinder(x):
    ch4_scale, ch4_o3, ch4_stwv, ch4_ari, ch4_aci_shape = x
    
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    f.fill_species_configs()
    f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = so2
    f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = bc
    f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = oc
    f.species_configs['baseline_emissions'].loc[dict(specie='VOC')] = voc
    f.species_configs['baseline_emissions'].loc[dict(specie='NOx')] = nox
    f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4[0]
    f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o
    f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2
    f.species_configs['baseline_concentration'].loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='Sulfur')] = so2
    f.emissions.loc[dict(specie='BC')] = bc
    f.emissions.loc[dict(specie='OC')] = oc
    f.emissions.loc[dict(specie='VOC')] = voc
    f.emissions.loc[dict(specie='NOx')] = nox

    f.concentration.loc[dict(specie='CH4')] = ch4[:, None, None]
    f.concentration.loc[dict(specie='N2O')] = n2o
    f.concentration.loc[dict(specie='CO2')] = co2
    f.concentration.loc[dict(specie='Equivalent effective stratospheric chlorine')] = 0

    f.temperature[:] = 0

    # pre-calibrated for aerosol-only runs from notebook 05: DON'T ADJUST
    for specie in species:
        f.species_configs['erfari_radiative_efficiency'].loc[dict(specie=specie)] = 0
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.00283793
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.01757433
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.0028512

    f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = -8.22336281e-01
    f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = 3.01888722e-02
    f.species_configs['aci_shape'].loc[dict(specie='BC')] = 5.07111186e-30
    f.species_configs['aci_shape'].loc[dict(specie='OC')] = 6.60381587e-16
    
    # these things we want to vary
    f.species_configs['forcing_scale'].loc[dict(specie="CH4")] = ch4_scale
    f.species_configs['ozone_radiative_efficiency'].loc[dict(specie="CH4")] = ch4_o3
    f.species_configs['h2o_stratospheric_factor'].loc[dict(specie="CH4")] = ch4_stwv
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie="CH4")] = ch4_ari
    f.species_configs['aci_shape'].loc[dict(specie="CH4")] = ch4_aci_shape
    
    f.run(progress=False)
    f_ch4_ch4 = f.forcing.values[-1,0,0,0] + f.forcing.values[-1,0,0,1]  # include N2O overlap as part of the methane forcing
    f_ch4_o3 = f.forcing.values[-1,0,0,9]
    f_ch4_stwv = f.forcing.values[-1,0,0,10]
    f_ch4_ari = f.forcing.values[-1,0,0,11]
    f_ch4_aci = f.forcing.values[-1,0,0,12]
    return np.array(
        [
            f_ch4_ch4 - target_ch4_ch4, 
            f_ch4_o3 - target_ch4_o3, 
            f_ch4_stwv - target_ch4_stwv, 
            f_ch4_ari - target_ch4_ari, 
            f_ch4_aci - target_ch4_aci, 
        ]
    )

In [18]:
solution = root(
    erf_rootfinder, 
    np.array(
        [
            1.11568,
            0.00012705,
            7/54,
            -5.093e-5,
            -0.00040355,
        ],
    ),
    method='lm',
    options={'maxiter': 500000}
)

In [19]:
solution.x

array([ 1.11547955e+00,  1.27049657e-04,  1.29652705e-01, -5.09377882e-05,
       -2.61186113e-04])

In [20]:
species
# Tunings to use in full calibration:
# ch4 1.11547955e+00
# o3 1.27049657e-04
# h2o 1.29652705e-01
# erfari -5.09377882e-05
# erfaci -2.61186113e-04

['CH4',
 'N2O',
 'CO2',
 'Equivalent effective stratospheric chlorine',
 'VOC',
 'NOx',
 'BC',
 'OC',
 'Sulfur',
 'Ozone',
 'Stratospheric water vapour',
 'Aerosol-radiation interactions',
 'Aerosol-cloud interactions']